## This One Uses a Football Icon
#

In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkintermapview import TkinterMapView
from PIL import Image, ImageTk
import json
import os
import atexit

stadiums = {
    "State Farm Stadium (Arizona Cardinals)": (33.5276, -112.2626),
    "Mercedes-Benz Stadium (Atlanta Falcons)": (33.755, -84.4009),
    "M&T Bank Stadium (Baltimore Ravens)": (39.2779, -76.6227),
    "Highmark Stadium (Buffalo Bills)": (42.7738, -78.786),
    "Bank of America Stadium (Carolina Panthers)": (35.2251, -80.8529),
    "Soldier Field (Chicago Bears)": (41.8625, -87.6166),
    "Paycor Stadium (Cincinnati Bengals)": (39.0955, -84.5161),
    "FirstEnergy Stadium (Cleveland Browns)": (41.5061, -81.6995),
    "AT&T Stadium (Dallas Cowboys)": (32.7473, -97.0945),
    "Empower Field at Mile High (Denver Broncos)": (39.7439, -105.0201),
    "Ford Field (Detroit Lions)": (42.3403, -83.0456),
    "Lambeau Field (Green Bay Packers)": (44.5013, -88.0622),
    "NRG Stadium (Houston Texans)": (29.6847, -95.4107),
    "Lucas Oil Stadium (Indianapolis Colts)": (39.7601, -86.1639),
    "TIAA Bank Field (Jacksonville Jaguars)": (30.3239, -81.6373),
    "Arrowhead Stadium (Kansas City Chiefs)": (39.049, -94.483),
    "Allegiant Stadium (Las Vegas Raiders)": (36.0908, -115.183),
    "SoFi Stadium (LA Rams/Chargers)": (33.9533, -118.339),
    "Hard Rock Stadium (Miami Dolphins)": (25.958, -80.2389),
    "U.S. Bank Stadium (Minnesota Vikings)": (44.974, -93.257),
    "Gillette Stadium (New England Patriots)": (42.0909, -71.2643),
    "Caesars Superdome (New Orleans Saints)": (29.9511, -90.0812),
    "MetLife Stadium (NY Giants/Jets)": (40.8135, -74.0745),
    "Lincoln Financial Field (Philadelphia Eagles)": (39.9008, -75.1675),
    "Heinz Field (Pittsburgh Steelers)": (40.4468, -80.0158),
    "Levi's Stadium (San Francisco 49ers)": (37.403, -121.97),
    "Lumen Field (Seattle Seahawks)": (47.5952, -122.3316),
    "Raymond James Stadium (Tampa Bay Buccaneers)": (27.9759, -82.5033),
    "Nissan Stadium (Tennessee Titans)": (36.1661, -86.7713),
    "FedExField (Washington Commanders)": (38.9076, -76.8645),
}

team_colors = {
    "Arizona Cardinals": "#97233F", "Atlanta Falcons": "#A71930", "Baltimore Ravens": "#241773",
    "Buffalo Bills": "#00338D", "Carolina Panthers": "#0085CA", "Chicago Bears": "#0B162A",
    "Cincinnati Bengals": "#FB4F14", "Cleveland Browns": "#311D00", "Dallas Cowboys": "#003594",
    "Denver Broncos": "#FB4F14", "Detroit Lions": "#0076B6", "Green Bay Packers": "#203731",
    "Houston Texans": "#03202F", "Indianapolis Colts": "#002C5F", "Jacksonville Jaguars": "#006778",
    "Kansas City Chiefs": "#E31837", "Las Vegas Raiders": "#000000", "LA Rams/Chargers": "#003594",
    "Miami Dolphins": "#008E97", "Minnesota Vikings": "#4F2683", "New England Patriots": "#002244",
    "New Orleans Saints": "#D3BC8D", "NY Giants/Jets": "#0B2265", "Philadelphia Eagles": "#004C54",
    "Pittsburgh Steelers": "#FFB612", "San Francisco 49ers": "#AA0000", "Seattle Seahawks": "#002244",
    "Tampa Bay Buccaneers": "#D50A0A", "Tennessee Titans": "#4B92DB", "Washington Commanders": "#5A1414",
}

DATA_FILE = "visited_stadiums_tk.json"

if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "r") as f:
        visited_stadiums = set(json.load(f))
else:
    visited_stadiums = set()

marker_icons = []
checkbox_vars = {}

# --------------------------
# FUNCTIONS
# --------------------------

def suppress_photomage_error():
    try:
        from PIL import ImageTk
        ImageTk.PhotoImage.__del__ = lambda self: None
    except Exception:
        pass

atexit.register(suppress_photomage_error)

def save_progress():
    with open(DATA_FILE, "w") as f:
        json.dump(list(visited_stadiums), f)
    messagebox.showinfo("Save", "Progress saved!")

def load_progress():
    global visited_stadiums
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            visited_stadiums = set(json.load(f))
        for stadium, var in checkbox_vars.items():
            var.set(stadium in visited_stadiums)
        update_map_markers()
        messagebox.showinfo("Load", "Progress loaded!")
    else:
        messagebox.showwarning("Load", "No saved progress found.")

def update_visited():
    visited_stadiums.clear()
    for stadium, var in checkbox_vars.items():
        if var.get():
            visited_stadiums.add(stadium)
    update_map_markers()

def update_map_markers():
    map_widget.delete_all_marker()
    marker_icons.clear()

    for stadium, (lat, lon) in stadiums.items():
        visited = stadium in visited_stadiums
        team_name = stadium.split('(')[-1].rstrip(')')
        color = team_colors.get(team_name, "gray")

        if visited:
            try:
                image = Image.open("football_icon.png").resize((24, 24))
                icon = ImageTk.PhotoImage(image)
                marker_icons.append(icon)
                map_widget.set_marker(lat, lon, text=stadium, icon=icon)
            except Exception as e:
                print(f"Error loading icon for {stadium}: {e}")
                map_widget.set_marker(lat, lon, text=stadium,
                                      marker_color_circle=color,
                                      marker_color_outside=color)
        else:
            map_widget.set_marker(lat, lon, text=stadium,
                                  marker_color_circle="gray",
                                  marker_color_outside="gray")

# --------------------------
# GUI
# --------------------------

root = tk.Tk()
root.title("NFL Stadium Tracker with Map")
root.geometry("1000x700")

frame_left = ttk.Frame(root)
frame_left.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

ttk.Label(frame_left, text="Select stadiums you've visited:", font=("Arial", 14)).pack(pady=5)

canvas = tk.Canvas(frame_left)
scrollbar = ttk.Scrollbar(frame_left, orient="vertical", command=canvas.yview)
scrollable_frame = ttk.Frame(canvas)

scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set, height=600)

canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Checkboxes
for stadium in stadiums.keys():
    var = tk.BooleanVar(value=(stadium in visited_stadiums))
    cb = ttk.Checkbutton(scrollable_frame, text=stadium, variable=var, command=update_visited)
    cb.pack(anchor='w', pady=2)
    checkbox_vars[stadium] = var

# Buttons
btn_frame = ttk.Frame(frame_left)
btn_frame.pack(pady=10)

ttk.Button(btn_frame, text="Save Progress", command=save_progress).grid(row=0, column=0, padx=5)
ttk.Button(btn_frame, text="Load Progress", command=load_progress).grid(row=0, column=1, padx=5)

# Map Frame
frame_right = ttk.Frame(root)
frame_right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

map_widget = TkinterMapView(frame_right, width=700, height=650)
map_widget.pack(fill=tk.BOTH, expand=True)

map_widget.set_position(39.8283, -98.5795)  # Center of USA
map_widget.set_zoom(4)

update_map_markers()

root.mainloop()

## This One Uses Colored Dots
#

In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkintermapview import TkinterMapView
from PIL import Image, ImageTk, ImageOps, ImageDraw
import json
import os
import atexit

# --- Data ---

stadiums = {
    "State Farm Stadium (Arizona Cardinals)": (33.5276, -112.2626),
    "Mercedes-Benz Stadium (Atlanta Falcons)": (33.755, -84.4009),
    "M&T Bank Stadium (Baltimore Ravens)": (39.2779, -76.6227),
    "Highmark Stadium (Buffalo Bills)": (42.7738, -78.786),
    "Bank of America Stadium (Carolina Panthers)": (35.2251, -80.8529),
    "Soldier Field (Chicago Bears)": (41.8625, -87.6166),
    "Paycor Stadium (Cincinnati Bengals)": (39.0955, -84.5161),
    "FirstEnergy Stadium (Cleveland Browns)": (41.5061, -81.6995),
    "AT&T Stadium (Dallas Cowboys)": (32.7473, -97.0945),
    "Empower Field at Mile High (Denver Broncos)": (39.7439, -105.0201),
    "Ford Field (Detroit Lions)": (42.3403, -83.0456),
    "Lambeau Field (Green Bay Packers)": (44.5013, -88.0622),
    "NRG Stadium (Houston Texans)": (29.6847, -95.4107),
    "Lucas Oil Stadium (Indianapolis Colts)": (39.7601, -86.1639),
    "TIAA Bank Field (Jacksonville Jaguars)": (30.3239, -81.6373),
    "Arrowhead Stadium (Kansas City Chiefs)": (39.049, -94.483),
    "Allegiant Stadium (Las Vegas Raiders)": (36.0908, -115.183),
    "SoFi Stadium (LA Rams/Chargers)": (33.9533, -118.339),
    "Hard Rock Stadium (Miami Dolphins)": (25.958, -80.2389),
    "U.S. Bank Stadium (Minnesota Vikings)": (44.974, -93.257),
    "Gillette Stadium (New England Patriots)": (42.0909, -71.2643),
    "Caesars Superdome (New Orleans Saints)": (29.9511, -90.0812),
    "MetLife Stadium (NY Giants/Jets)": (40.8135, -74.0745),
    "Lincoln Financial Field (Philadelphia Eagles)": (39.9008, -75.1675),
    "Heinz Field (Pittsburgh Steelers)": (40.4468, -80.0158),
    "Levi's Stadium (San Francisco 49ers)": (37.403, -121.97),
    "Lumen Field (Seattle Seahawks)": (47.5952, -122.3316),
    "Raymond James Stadium (Tampa Bay Buccaneers)": (27.9759, -82.5033),
    "Nissan Stadium (Tennessee Titans)": (36.1661, -86.7713),
    "FedExField (Washington Commanders)": (38.9076, -76.8645),
}

team_colors = {
    "Arizona Cardinals": "#97233F", "Atlanta Falcons": "#A71930", "Baltimore Ravens": "#241773",
    "Buffalo Bills": "#00338D", "Carolina Panthers": "#0085CA", "Chicago Bears": "#0B162A",
    "Cincinnati Bengals": "#FB4F14", "Cleveland Browns": "#311D00", "Dallas Cowboys": "#003594",
    "Denver Broncos": "#FB4F14", "Detroit Lions": "#0076B6", "Green Bay Packers": "#203731",
    "Houston Texans": "#03202F", "Indianapolis Colts": "#002C5F", "Jacksonville Jaguars": "#006778",
    "Kansas City Chiefs": "#E31837", "Las Vegas Raiders": "#000000", "LA Rams/Chargers": "#003594",
    "Miami Dolphins": "#008E97", "Minnesota Vikings": "#4F2683", "New England Patriots": "#002244",
    "New Orleans Saints": "#D3BC8D", "NY Giants/Jets": "#0B2265", "Philadelphia Eagles": "#004C54",
    "Pittsburgh Steelers": "#FFB612", "San Francisco 49ers": "#AA0000", "Seattle Seahawks": "#002244",
    "Tampa Bay Buccaneers": "#D50A0A", "Tennessee Titans": "#4B92DB", "Washington Commanders": "#5A1414",
}

DATA_FILE = "visited_stadiums_tk.json"
visited_stadiums = set()
marker_icons = []
checkbox_vars = {}

# --------------------------
# FUNCTIONS
# --------------------------

def suppress_photomage_error():
    try:
        from PIL import ImageTk
        ImageTk.PhotoImage.__del__ = lambda self: None
    except Exception:
        pass

atexit.register(suppress_photomage_error)

def save_progress():
    with open(DATA_FILE, "w") as f:
        json.dump(list(visited_stadiums), f)
    messagebox.showinfo("Save", "Progress saved!")

def load_progress():
    global visited_stadiums
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            visited_stadiums = set(json.load(f))
        for stadium, var in checkbox_vars.items():
            var.set(stadium in visited_stadiums)
        update_map_markers()
        messagebox.showinfo("Load", "Progress loaded!")
    else:
        messagebox.showwarning("Load", "No saved progress found.")

def update_visited():
    visited_stadiums.clear()
    for stadium, var in checkbox_vars.items():
        if var.get():
            visited_stadiums.add(stadium)
    update_map_markers()

def create_colored_icon(border_color):
    icon_path = os.path.join(os.path.dirname(__file__), "football_icon.png")
    base = Image.open(icon_path).convert("RGBA").resize((28, 28), Image.ANTIALIAS)

    border_size = 2
    new_size = (base.size[0] + border_size*2, base.size[1] + border_size*2)
    bordered = Image.new("RGBA", new_size, (0, 0, 0, 0))

    # Draw border rectangle
    draw = ImageDraw.Draw(bordered)
    draw.rectangle([0, 0, new_size[0]-1, new_size[1]-1], outline=border_color, width=border_size)

    # Paste base icon inside border
    bordered.paste(base, (border_size, border_size), base)

    return ImageTk.PhotoImage(bordered)

def create_small_gray_dot():
    dot = Image.new("RGBA", (6, 6), (0, 0, 0, 0))
    draw = ImageDraw.Draw(dot)
    draw.ellipse((1, 1, 5, 5), fill="gray")
    return ImageTk.PhotoImage(dot)

def update_map_markers():
    map_widget.delete_all_marker()
    marker_icons.clear()

    for stadium, (lat, lon) in stadiums.items():
        visited = stadium in visited_stadiums
        team_name = stadium.split('(')[-1].rstrip(')')
        color = team_colors.get(team_name, "gray")

        if visited:
            try:
                icon = create_colored_icon(color)
                marker_icons.append(icon)
                map_widget.set_marker(lat, lon, text=stadium, icon=icon)
            except Exception as e:
                print(f"Error loading icon for {stadium}: {e}")
                map_widget.set_marker(lat, lon, text=stadium,
                                      marker_color_circle=color,
                                      marker_color_outside=color)
        else:
            dot = create_small_gray_dot()
            marker_icons.append(dot)
            # No text label for unvisited stadium
            map_widget.set_marker(lat, lon, text=None, icon=dot)

# --------------------------
# GUI
# --------------------------

root = tk.Tk()
root.title("NFL Stadium Tracker with Map")
root.geometry("1000x700")

# --- Left panel
frame_left = ttk.Frame(root)
frame_left.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

ttk.Label(frame_left, text="Select stadiums you've visited:", font=("Arial", 14)).pack(pady=5)

canvas = tk.Canvas(frame_left)
scrollbar = ttk.Scrollbar(frame_left, orient="vertical", command=canvas.yview)
scrollable_frame = ttk.Frame(canvas)

scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set, height=600)

canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Checkboxes
for stadium in stadiums.keys():
    var = tk.BooleanVar(value=(stadium in visited_stadiums))
    cb = ttk.Checkbutton(scrollable_frame, text=stadium, variable=var, command=update_visited)
    cb.pack(anchor='w', pady=2)
    checkbox_vars[stadium] = var

# Buttons
btn_frame = ttk.Frame(frame_left)
btn_frame.pack(pady=10)

ttk.Button(btn_frame, text="Save Progress", command=save_progress).grid(row=0, column=0, padx=5)
ttk.Button(btn_frame, text="Load Progress", command=load_progress).grid(row=0, column=1, padx=5)

# Map Frame
frame_right = ttk.Frame(root)
frame_right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

map_widget = TkinterMapView(frame_right, width=700, height=650)
map_widget.pack(fill=tk.BOTH, expand=True)

map_widget.set_position(39.8283, -98.5795)  # Center of USA
map_widget.set_zoom(4)

update_map_markers()

root.mainloop()

Error loading icon for M&T Bank Stadium (Baltimore Ravens): name '__file__' is not defined
Error loading icon for M&T Bank Stadium (Baltimore Ravens): name '__file__' is not defined
Error loading icon for Bank of America Stadium (Carolina Panthers): name '__file__' is not defined
Error loading icon for M&T Bank Stadium (Baltimore Ravens): name '__file__' is not defined
Error loading icon for Highmark Stadium (Buffalo Bills): name '__file__' is not defined
Error loading icon for Bank of America Stadium (Carolina Panthers): name '__file__' is not defined
Error loading icon for M&T Bank Stadium (Baltimore Ravens): name '__file__' is not defined
Error loading icon for Highmark Stadium (Buffalo Bills): name '__file__' is not defined
Error loading icon for Bank of America Stadium (Carolina Panthers): name '__file__' is not defined
Error loading icon for Empower Field at Mile High (Denver Broncos): name '__file__' is not defined
Error loading icon for M&T Bank Stadium (Baltimore Ravens): name